In [ ]:
import os
from pathlib import Path
import yaml
import pandas as pd
import seaborn as sns

In [ ]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.set_option("display.max_colwidth", None)
sns.set()

In [ ]:
os.environ["APP_ROOT"] = str(Path().cwd().parent.parent)
os.environ["DATA_DIR"] = str(Path().cwd().parent.parent / "data")

In [ ]:
cities = yaml.safe_load((Path(os.getenv("APP_ROOT")) / "config.yaml").open())["cities"]

In [ ]:
def summarize(current_city):
    # fmt: off
    df = (
        pd.read_json(f"{os.getenv('DATA_DIR')}/cost_of_living_cleansed.json")
        [[
            "city",
            "country",
            'Apartment (1 bedroom) Outside of Centre',
            'Apartment (1 bedroom) in City Centre',
            'Apartment (3 bedrooms) Outside of Centre',
            'Apartment (3 bedrooms) in City Centre',
            'Average Monthly Net Salary (After Tax)',
            'Basic (Electricity, Heating, Cooling, Water, Garbage) for 85m2 Apartment',
            'Cappuccino (regular)',
            "Cigarettes 20 Pack (Marlboro)",
            "Mobile Phone Monthly Plan with Calls and 10GB+ Data",
            'McMeal at McDonalds (or Equivalent Combo Meal)',
            'Price per Square Meter to Buy Apartment Outside of Centre',
            'Price per Square Meter to Buy Apartment in City Centre',
            'Water (1.5 liter bottle)',
        ]]
        .merge(
            pd.read_json(f"{os.getenv('DATA_DIR')}/happiness.json")
            .rename(columns={"Country": "country"})
            .drop(columns=["Year", "Happiness.Rank"]),
            on=["country"],
            how="left",
        )
        .melt(id_vars=["city", "country"], var_name="feature", value_name="value")
    )
    # fmt: on

    current_city_val_name = "value_in_current_city"
    current_df = (
        df[df["city"] == current_city].drop(columns=["city", "country"]).rename(columns={"value": current_city_val_name})
    )

    rest_df = df[df["city"] != current_city]

    diff_df = rest_df.merge(current_df, on="feature", how="left")
    diff_df["diff_amount"] = diff_df["value"] - diff_df[current_city_val_name]
    diff_df["diff_rate"] = round(diff_df["value"] / diff_df[current_city_val_name] - 1, 2)
    diff_df = diff_df[["city", "feature", current_city_val_name, "diff_amount", "diff_rate"]]

    output_df = rest_df.merge(diff_df, on=["city", "feature"], how="left")
    output_df.to_json(
        f"{os.getenv('DATA_DIR')}/summary_{current_city.lower().replace("-", "_")}.json",
        orient="records",
        indent=2,
    )

In [ ]:
for city in cities:
    summarize(city)